In [32]:
import pandas as pd 

import warnings
warnings.filterwarnings("ignore")

Auto = pd.read_csv('data/Auto.csv',na_values = "?").dropna(inplace=True)
Auto['orgn1'] = 0
Auto.orgn1[Auto['origin']==1] = 1
Auto['orgn2'] = 0
Auto.orgn2[Auto['origin']==2] = 1
Median = Auto['mpg'].median()
Auto['mpg_high'] = 0
Auto.mpg_high[Auto['mpg']>=Median] = 1

In [35]:
#a 
import time 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from statistics import mean, stdev

y = Auto['mpg_high'].values
X = Auto[['cylinders', 'displacement', 'horsepower', 'weight','acceleration', 'year', 'orgn1', 'orgn2']].values
time_start1 = time.clock()
MSE = []
for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state=i+100)
    LogReg = LogisticRegression().fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_now = (y_test == y_pred).mean()
    MSE.append(MSE_now)
MSE_avg = mean(MSE)
time_end1 = time.clock()
print('The average MSE is', MSE_avg)
print('This computation takes',time_end1-time_start1,'seconds')

The average MSE is 0.8976086956521739
This computation takes 0.18068643399965367 seconds


In [37]:
#b
import multiprocessing
from dask import compute, delayed
import dask.multiprocessing

def doublecheck(bootstrap, seed, Data):
    X, y = Data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.35, random_state=seed+bootstrap)
    LogReg = LogisticRegression().fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    return (y_test == y_pred).mean()
time_start2 = time.clock()
MSE_2 = []
for i in range(100):
    MSE_2.append(delayed(doublecheck)(i, 100, [X, y]))
MSE2_avg = delayed(mean)(MSE_2)
MSE2_avg = compute(MSE2_avg, num_workers=num_cores)[0]
time_end2 = time.clock()
print('The average MSE is', MSE2_avg)
print('This computation takes',time_end2-time_start2,'seconds')

The average MSE is 0.8976086956521739
This computation takes 0.17561892099911347 seconds
